#### 개요
온라인으로 데이터를 받아, 데이터를 전처리 할 수 있도록 공사비 단위로 하나의 레코드로 취합하는 역활을 수행함
* STEP01: 학습데이터에서 특정 데이터를 추출해 온라인 입력형태의 데이터 샘플 생성`(이 작업은 실제 운영환경에서 실행되지 않음)`
* STEP02: JSON 데이터를 이용해 공사비별 전주/전선/인입선 갯 수 계산

In [11]:
import os
import sys
import json
import pandas as pd
from collections import Counter

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data, read_data
# from aidd.online.data_manager import get_online_sample

In [2]:
pdata = get_provide_data()

[f847780a7582][2024-03-13 16:06:59.297808] 제공받은 데이터 불러오기 시작
[f847780a7582][2024-03-13 16:07:26.050503]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:26.752620
[f847780a7582][2024-03-13 16:07:49.922624]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:23.872003
[f847780a7582][2024-03-13 16:08:19.885009]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:29.962266
[f847780a7582][2024-03-13 16:08:32.220275]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:12.335145
[f847780a7582][2024-03-13 16:08:32.220365] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:32.922560


#### STEP01: 온라인 시험 대상 추출

In [7]:
def get_online_sample(data):
    """온라인 서비스 샘플 데이터 생성 함수
    * 온라인 서비스나 온라인 서비스를 위한 전처리 프로세스 테스트를 위한
      샘플 데이터를 생성하는 함수로, 대상 데이터는 `config`에 지정되어 있음
    * 실제 온라인 작업에선 사용하지 않음
    Args:
        data (dict): 학습용으로 제공받은 전체 데이터
            - `key: df`로 구성된 딕셔너리로 제공됨
    Returns:
        jsons (dict): Online으로 받을 예정인 JSON 데이터의 딕셔너리
    """
    # 전체 데이터에서 샘플 데이터만 추출한 데이터프레임 딕셔너리
    jsons = {}
    for key in cfg.DATA_TYPE:
        df = data[key]
        df = df[df.CONS_ID.isin(cfg.CHECK_CONS_IDS)]
        # 공사비 데이터셋에 있는 총공사비를 제거하는 코드(3줄이나 소요됨;;)
        modeling_cols = cfg.MODELING_COLS[key]
        if 'TOTAL_CONS_COST' in modeling_cols:
            modeling_cols.remove('TOTAL_CONS_COST')
        jsons[key] = df[modeling_cols].to_json(orient='records')
        
    return jsons

In [14]:
json_dict = get_online_sample(data=pdata)

#### STEP02: 공사비별 전주/전선/인입선 갯 수 계산
* 'RET_MSG' 추가: 예측 결과값과 함께 리턴하는 코드
  * 전주 갯 수가 0 또는 10개 이상인 경우: 'POLE_ERROR'
  * 전선 갯 수가 0 또는 10개 이상인 경우: 'LINE_ERROR'
  * 정상인 경우: 'OK'

In [28]:
# 이 부분은 추후에 수정할 사항이 많아 보임
# RESTful로 입력받은 json데이터를 데이터프레임으로 변환

# 입력된 데이터를 데이터프레임으로 변환한 딕셔너리
# 모델에 사용할 데이터 딕셔너리
mdict = {}
for key in json_dict.keys():
    mdict[key] = pd.read_json(json_dict[key])
# 총공사비가 없는 경우에는 총공사비 컬럼을 0으로 추가
# 예측해야할 값이긴 하지만 온라인/배치 전처리를 같은 함수로 하기 위해 
# 컬럼만 추가함
if 'TOTAL_CONS_COST' not in mdict['CONS'].keys():
    mdict['CONS']['TOTAL_CONS_COST'] = 0

In [29]:
# (참조용)데이터 셋 별 공사대상 레코드/컬럼 크기 확인
for key in cfg.DATA_TYPE:
    print(f'{key}: {mdict[key].shape}')

CONS: (3, 8)
POLE: (9, 6)
LINE: (9, 11)
SL: (3, 6)


#### STEP02: 공사비별 전주/전선/인입선 갯 수 계산
* 배치 작업에서 사용하는 함수와 동일